In [1]:
# Importing modules
import pandas as pd
import os
os.chdir('..')
# Read data into papers
papers = pd.read_csv(r"C:\Users\Test\Documents\FLTextRawData.csv")
# Print head
papers.head()

C:\Users\Test\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Property Id,Date_of_stay,Review,Rating,Chain Segment,Walkability,Avg Value Rating,Property Type,Location Type
0,1691,7/31/2022,"This hotel is very dirty, the doors are not pr...",1.0,Upper Midscale,Car recommended,1.5,Limited-Service Hotel,Mid-sized City - Urban
1,1691,6/30/2022,This place is a flop house! Do not stay here! ...,1.0,Upper Midscale,Car recommended,1.5,Limited-Service Hotel,Mid-sized City - Urban
2,1691,9/30/2022,Very disappointed with quality and cleanliness...,1.0,Upper Midscale,Car recommended,1.5,Limited-Service Hotel,Mid-sized City - Urban
3,1691,6/30/2022,"Weekly rate, the room was accommodating, frien...",4.0,Upper Midscale,Car recommended,1.5,Limited-Service Hotel,Mid-sized City - Urban
4,1691,9/30/2022,The high quality stands I was used to in La Qu...,1.0,Upper Midscale,Car recommended,1.5,Limited-Service Hotel,Mid-sized City - Urban


In [2]:
# Load the regular expression library
import re
# Remove punctuation
papers['paper_text_processed'] = \
papers['Review'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
# Convert the titles to lowercase
papers['paper_text_processed'] = \
papers['paper_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['paper_text_processed'].head()

0    this hotel is very dirty the doors are not pro...
1    this place is a flop house do not stay here i ...
2    very disappointed with quality and cleanliness...
3    weekly rate the room was accommodating friendl...
4    the high quality stands i was used to in la qu...
Name: paper_text_processed, dtype: object

In [3]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'hotel', 'stay'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Test\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['dirty', 'doors', 'properly', 'sealed', 'bugs', 'get', 'rooms', 'switch', 'rooms', 'three', 'times', 'every', 'room', 'smelled', 'marijuana', 'ants', 'bathrooms', 'rooms', 'clean', 'city', 'people', 'parking', 'lot', 'hours', 'night', 'would', 'recommend']


In [4]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 3), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)]


In [12]:
from pprint import pprint
# number of topics
num_topics = 6
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.017*"good" + 0.015*"staff" + 0.015*"clean" + 0.014*"breakfast" + '
  '0.014*"room" + 0.014*"nice" + 0.011*"great" + 0.011*"area" + 0.010*"rooms" '
  '+ 0.010*"friendly"'),
 (1,
  '0.019*"staff" + 0.014*"great" + 0.013*"service" + 0.012*"room" + 0.012*"us" '
  '+ 0.007*"would" + 0.007*"time" + 0.007*"beach" + 0.007*"made" + '
  '0.007*"friendly"'),
 (2,
  '0.013*"pool" + 0.010*"room" + 0.009*"one" + 0.008*"service" + 0.007*"staff" '
  '+ 0.006*"us" + 0.006*"area" + 0.005*"people" + 0.005*"front" + '
  '0.005*"disney"'),
 (3,
  '0.028*"room" + 0.011*"us" + 0.010*"would" + 0.008*"get" + 0.008*"night" + '
  '0.008*"day" + 0.007*"one" + 0.007*"desk" + 0.006*"check" + 0.006*"front"'),
 (4,
  '0.027*"room" + 0.007*"one" + 0.007*"would" + 0.006*"nice" + 0.006*"rooms" + '
  '0.006*"bathroom" + 0.006*"like" + 0.006*"get" + 0.005*"pool" + '
  '0.005*"staff"'),
 (5,
  '0.021*"great" + 0.017*"pool" + 0.016*"beach" + 0.011*"room" + 0.010*"nice" '
  '+ 0.008*"staff" + 0.007*"resort" + 0.007

In [13]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis 
import pickle 
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('C:/Users/Test/Documents/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, 'C:/Users/Test/Documents/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

C:\Users\Test\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.064111 -0.020504       1        1  21.270893
3     -0.052775  0.011439       2        1  19.203844
0      0.020976 -0.066080       3        1  18.683414
1      0.047955  0.062985       4        1  17.555409
4     -0.073811 -0.005380       5        1  15.780136
2     -0.006455  0.017540       6        1   7.506304, topic_info=       Term           Freq          Total Category  logprob  loglift
170   beach  228562.000000  228562.000000  Default  30.0000  30.0000
61     pool  359440.000000  359440.000000  Default  29.0000  29.0000
17     room  742167.000000  742167.000000  Default  28.0000  28.0000
236   great  457049.000000  457049.000000  Default  27.0000  27.0000
66       us  258558.000000  258558.000000  Default  26.0000  26.0000
...     ...            ...            ...      ...      ...      ...
7     every    9599.624941   94460.871541   Topic6  -5.8108   0.3030
236   great   11819.228862  457049.923188   Topic6  -5.6028  -1.0656
235    good   10794.010911  280616.901962   Topic6  -5.6936  -0.6686
18    rooms   10543.996249  268938.628373   Topic6  -5.7170  -0.6495
1851    bar    9818.070509  120033.450120   Topic6  -5.7883   0.0859

[675 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
905        1  0.048628       ac
905        2  0.191204       ac
905        3  0.099006       ac
905        4  0.040750       ac
905        5  0.579446       ac
...      ...       ...      ...
691        3  0.094345     year
691        4  0.302253     year
691        5  0.076235     year
691        6  0.158541     year
74029      4  0.987104  yolanda

[2220 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 1, 2, 5, 3])